# MAT017 - Cálculo
## Trabalho - parte 3
Resolvendo a distribuição de weibus de probabilidade utilizando o **Método de Monte Carlo**.

Autores:
- Fernando Gomes
- Helena Barboza

## Problema
A distribuição de Weibull é frequentemente usada para modelar a vida útil de produtos, tempo até a falha de equipamentos, entre outros fenômenos.

$$
f(x; \lambda, k) = \frac{k}{\lambda} \left( \frac{x}{\lambda} \right)^{k-1} \exp \left[ - \left( \frac{x}{\lambda} \right)^k \right]
$$

Foram escolhidos os seguintes valores para os parâmetros da distribuição de Weibull:
- Exemplo de Aplicação: Modelagem da vida útil de lâmpadas.
    - Tempo médio até a falha (parâmetro de escala, $λ$): 1500.0 horas.
    - Fator de forma ($k$): 1.8.

In [1]:
!pip install pymc numpy arviz ipywidgets

In [2]:
import numpy as np
import pymc as pm
import math

# função da distribuição de Weibull (f.d.p)
def weibull_fdp(x, lambda_val, k):
    return (k / lambda_val) * (x / lambda_val)**(k - 1) * np.exp(-(x / lambda_val)**k)



WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
# Parâmetros da distribuição de Weibull
lambda_val = 1500.0  # Tempo médio até a falha em horas
k = 1.8             # Fator de forma

# Intervalo de integração escolhido
base = 3000.0

# Altura da pdf da Weibull em base
altura = weibull_fdp(base, lambda_val, k)

In [4]:
# Definção do modelo
with pm.Model() as model:
    X = pm.Uniform("X", 0.1, base, shape=3000)
    Y = pm.Uniform("Y", 0, altura, shape=3000)

    trace = pm.sample()

amostrasX = trace.posterior["X"].values[1][999]
amostrasY = trace.posterior["Y"].values[1][999]

pedras = list(zip(amostrasX, amostrasY))

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [X, Y]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 31 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


In [5]:

# Calculo da integral usando Monte Carlo
ct = 0
for (x, y) in pedras:
    if y <= weibull_fdp(x, lambda_val, k):
        ct += 1

# Calcular área estimada sob a curva
area = (base - 0.1) * altura * ct / len(pedras)
print(f"Área estimada: {area}")

Área estimada: 0.19170519321253296
